In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

In [2]:
# My assets
assets = ["NIFTYBEES.NS", "SILVERIETF.NS", "GOLDBEES.NS", "BANKBEES.NS"]

# My invested amounts (₹)
invested_amounts = {
    "NIFTYBEES.NS": 254224,
    "SILVERIETF.NS": 63175,
    "GOLDBEES.NS": 91636,
    "BANKBEES.NS": 75419
}

# Settings (same as portfolio_calc.py)
settings = {
    "period": "5y",
    "interval": "1wk",
    "price_field": "Adj Close",
    "annualization_factor": 52,
    "market_ticker": "^NSEI",
    "rf_annual": 0.0,
    "use_excess_returns_for_beta": False
}

### Weights w (the portfolio recipe)

In [3]:
amounts = pd.Series(invested_amounts, dtype="float64").reindex(assets)
total_invested = float(amounts.sum())
w = (amounts / total_invested)

total_invested, w

(484454.0,
 NIFTYBEES.NS     0.524764
 SILVERIETF.NS    0.130405
 GOLDBEES.NS      0.189153
 BANKBEES.NS      0.155678
 dtype: float64)

Explanation:
- w is the fraction of your money in each asset.
- These weights are what you plug into:


\sigma_p^2 = w^T \Sigma w,\quad \beta_p = w^T \beta

### Download weekly prices (what download_prices() does)

In [4]:
def download_prices(tickers, period, interval, price_field):
    df = yf.download(
        tickers=tickers,
        period=period,
        interval=interval,
        group_by="ticker",
        auto_adjust=False,
        progress=False
    )
    prices = pd.DataFrame({t: df[t][price_field] for t in tickers}).sort_index()
    prices = prices.dropna(how="any")
    if prices.empty:
        raise ValueError("No price data after cleaning.")
    return prices

prices = download_prices(assets, settings["period"], settings["interval"], settings["price_field"])
prices.tail()

,NIFTYBEES.NS,SILVERIETF.NS,GOLDBEES.NS,BANKBEES.NS
Date,,,,
2025-12-22,294.459991,229.729996,114.309998,608.289978
2025-12-29,297.549988,231.679993,111.820000,619.909973
2026-01-05,290.589996,240.399994,113.610001,611.830017
2026-01-12,291.040009,281.670013,117.779999,619.919983
2026-01-19,286.279999,313.750000,125.860001,612.169983


### Returns (what compute_covariance() starts with)

In [5]:
use_log_returns = False

if use_log_returns:
    rets = np.log(prices / prices.shift(1)).dropna()
else:
    rets = prices.pct_change().dropna()

rets.tail()

,NIFTYBEES.NS,SILVERIETF.NS,GOLDBEES.NS,BANKBEES.NS
Date,,,,
2025-12-22,0.003444,0.142424,0.040885,-0.002324
2025-12-29,0.010494,0.008488,-0.021783,0.019103
2026-01-05,-0.023391,0.037638,0.016008,-0.013034
2026-01-12,0.001549,0.171672,0.036704,0.013223
2026-01-19,-0.016355,0.113892,0.068602,-0.012502


### Covariance matrix \Sigma + portfolio variance w^T\Sigma w

In [6]:
cov_weekly = rets.cov()
cov_annual = cov_weekly * settings["annualization_factor"]
corr = rets.corr()

var_weekly = float(w.T @ cov_weekly.reindex(index=assets, columns=assets) @ w)
vol_weekly = float(np.sqrt(var_weekly))

var_annual = float(w.T @ cov_annual.reindex(index=assets, columns=assets) @ w)
vol_annual = float(np.sqrt(var_annual))

cov_weekly, var_weekly, vol_weekly, var_annual, vol_annual

(               NIFTYBEES.NS  SILVERIETF.NS  GOLDBEES.NS  BANKBEES.NS
 NIFTYBEES.NS       0.000273       0.000031    -0.000007     0.000296
 SILVERIETF.NS      0.000031       0.001444     0.000544     0.000047
 GOLDBEES.NS       -0.000007       0.000544     0.000376    -0.000007
 BANKBEES.NS        0.000296       0.000047    -0.000007     0.000438,
 0.0002031529689556317,
 0.014253173995837969,
 0.010563954385692848,
 0.1027810993602075)

### Download market returns (NIFTY 50) for beta

In [7]:
mkt_ticker = settings["market_ticker"]
mkt_prices = download_prices([mkt_ticker], settings["period"], settings["interval"], settings["price_field"])

if use_log_returns:
    mkt_rets = np.log(mkt_prices[mkt_ticker] / mkt_prices[mkt_ticker].shift(1)).dropna()
else:
    mkt_rets = mkt_prices[mkt_ticker].pct_change().dropna()

mkt_rets.tail()

Date
2025-12-22    0.002923
2025-12-29    0.010992
2026-01-05   -0.024508
2026-01-12    0.000430
2026-01-19   -0.017975
Name: ^NSEI, dtype: float64

### Risk-free handling (optional CAPM excess returns)

In [17]:
# Turn ON CAPM excess-return beta
settings["use_excess_returns_for_beta"] = True

# Set annual risk-free rate (decimal). Example: 7% => 0.07
settings["rf_annual"] = 0.07

ann_factor = float(settings["annualization_factor"])  # 52 for weekly

rf_annual = float(settings["rf_annual"])

# Convert annual RF to weekly RF (compound-consistent)
rf_period = (1.0 + rf_annual) ** (1.0 / ann_factor) - 1.0

# Excess returns (asset & market)
rets_for_beta = rets.sub(rf_period)
mkt_for_beta = mkt_rets.sub(rf_period)

rf_period, rf_period * 100

(0.0013019746893534467, 0.13019746893534467)

### Individual betas (Cov/Var definition)

In [18]:
def compute_betas(asset_returns: pd.DataFrame, market_returns: pd.Series) -> pd.Series:
    aligned = asset_returns.join(market_returns.rename("MKT"), how="inner")
    mkt = aligned["MKT"]
    var_m = float(mkt.var(ddof=1))
    if var_m == 0:
        raise ValueError("Market variance is zero.")
    betas = {}
    for col in asset_returns.columns:
        betas[col] = float(aligned[col].cov(mkt) / var_m)
    return pd.Series(betas)

rets_for_beta = rets_for_beta.reindex(columns=assets)
betas = compute_betas(rets_for_beta, mkt_for_beta)

betas

NIFTYBEES.NS     0.944659
SILVERIETF.NS    0.127045
GOLDBEES.NS     -0.006642
BANKBEES.NS      1.027793
dtype: float64

### Portfolio beta (two equivalent methods)

In [19]:
# Method 1: weighted average of asset betas
beta_port_weighted = float((w * betas.reindex(assets)).sum())

# Method 2: compute beta from portfolio returns vs market
port_rets = (rets_for_beta[assets] * w.values).sum(axis=1)
aligned_pm = port_rets.to_frame("PORT").join(mkt_for_beta.rename("MKT"), how="inner")
beta_port_from_returns = float(aligned_pm["PORT"].cov(aligned_pm["MKT"]) / aligned_pm["MKT"].var(ddof=1))

beta_port_weighted, beta_port_from_returns

(0.6710386652092278, 0.6710386652092278)

In [11]:
# Raw beta
beta_raw = float(aligned_pm["PORT"].cov(aligned_pm["MKT"]) / aligned_pm["MKT"].var(ddof=1))

# Subtract constants (means) from both series
PORT_tilde = aligned_pm["PORT"] - aligned_pm["PORT"].mean()
MKT_tilde = aligned_pm["MKT"] - aligned_pm["MKT"].mean()

beta_tilde = float(PORT_tilde.cov(MKT_tilde) / MKT_tilde.var(ddof=1))

beta_raw, beta_tilde, beta_raw - beta_tilde

(0.6710386652092278, 0.6710386652092278, 0.0)

### risk decomposition (systematic vs idiosyncratic variance)

In [12]:
beta = beta_port_weighted

sigma_p2 = float(port_rets.var(ddof=1))      # weekly portfolio variance from return series
sigma_m2 = float(mkt_for_beta.var(ddof=1))   # weekly market variance

systematic_var = (beta**2) * sigma_m2
idiosyncratic_var = max(0.0, sigma_p2 - systematic_var)

pct_systematic = systematic_var / sigma_p2 if sigma_p2 > 0 else np.nan
pct_idio = idiosyncratic_var / sigma_p2 if sigma_p2 > 0 else np.nan

systematic_var, idiosyncratic_var, pct_systematic, pct_idio

(0.0001620681195421599,
 4.108484941347184e-05,
 0.7977639725145012,
 0.20223602748549885)

### Hedge sizing with NIFTY futures (your practical step)

In [13]:
V_P = 4084000          # your portfolio value (₹)
beta_P = beta_port_weighted
beta_T = 0.30          # example: partial hedge target beta
lot_size = 65          # you said 65

# You must plug the LIVE/RECENT NIFTY futures price here:
futures_price = 25500  # <-- replace with current NIFTY50 futures price you are hedging

F = futures_price * lot_size
n = (beta_T - beta_P) * (V_P / F)

n

-0.9142213627236722

In [14]:
n_round = int(np.round(n))   # nearest integer
n_floor = int(np.floor(n))   # more conservative (less hedge)
n_ceil  = int(np.ceil(n))    # more aggressive (more hedge)

n, n_round, n_floor, n_ceil

(-0.9142213627236722, -1, -1, 0)

In [15]:
def resulting_beta(beta_P, V_P, F, n_contracts):
    # each contract changes beta by approx (F/V_P) with sign depending on short/long
    # if you SHORT, n_contracts is negative
    return beta_P + (n_contracts * F / V_P)

beta_after_round = resulting_beta(beta_P, V_P, F, n_round)
beta_after_floor = resulting_beta(beta_P, V_P, F, n_floor)
beta_after_ceil  = resulting_beta(beta_P, V_P, F, n_ceil)

beta_P, beta_after_round, beta_after_floor, beta_after_ceil

(0.6710386652092278,
 0.26518655943057945,
 0.26518655943057945,
 0.6710386652092278)

In [16]:
summary = {
    "total_invested": total_invested,
    "weights": w,
    "var_weekly": var_weekly,
    "vol_weekly": vol_weekly,
    "var_annual": var_annual,
    "vol_annual": vol_annual,
    "betas": betas,
    "beta_port_weighted": beta_port_weighted,
    "beta_port_from_returns": beta_port_from_returns,
    "pct_systematic_var": pct_systematic,
    "pct_idiosyncratic_var": pct_idio
}
summary

{'total_invested': 484454.0,
 'weights': NIFTYBEES.NS     0.524764
 SILVERIETF.NS    0.130405
 GOLDBEES.NS      0.189153
 BANKBEES.NS      0.155678
 dtype: float64,
 'var_weekly': 0.0002031529689556317,
 'vol_weekly': 0.014253173995837969,
 'var_annual': 0.010563954385692848,
 'vol_annual': 0.1027810993602075,
 'betas': NIFTYBEES.NS     0.944659
 SILVERIETF.NS    0.127045
 GOLDBEES.NS     -0.006642
 BANKBEES.NS      1.027793
 dtype: float64,
 'beta_port_weighted': 0.6710386652092278,
 'beta_port_from_returns': 0.6710386652092278,
 'pct_systematic_var': 0.7977639725145012,
 'pct_idiosyncratic_var': 0.20223602748549885}

## Calculating Performance

In [21]:

# 1) Portfolio weekly return series
rp = (rets[assets] * w.values).sum(axis=1)

# 2) Align portfolio + market on same weeks (important)
aligned = rp.to_frame("RP").join(mkt_rets.rename("RM"), how="inner").dropna()
rp_a = aligned["RP"]
rm_a = aligned["RM"]

# 3) Excess returns (weekly)
rp_excess = rp_a - rf_period
rm_excess = rm_a - rf_period

# 4) Average excess return (weekly)
mean_excess_weekly = float(rp_excess.mean())

# 5) Portfolio volatility (weekly)
sigma_p_weekly = float(rp_a.std(ddof=1))

# 6) Sharpe (weekly)
sharpe_weekly = mean_excess_weekly / sigma_p_weekly

# Common: Annualized Sharpe (using sqrt(time))
sharpe_annual = sharpe_weekly * np.sqrt(ann_factor)

# 7) Treynor (weekly) - uses portfolio beta
beta_p = float(beta_port_weighted)
treynor_weekly = mean_excess_weekly / beta_p

# Annualized Treynor (approx): multiply mean by 52 (beta unchanged)
treynor_annual = (mean_excess_weekly * ann_factor) / beta_p

# 8) M² (Modigliani–Modigliani)
# Benchmark volatility = market volatility (weekly)
sigma_m_weekly = float(rm_a.std(ddof=1))

# M² weekly in return units:
# M2 = rf + Sharpe * sigma_benchmark
m2_weekly = rf_period + sharpe_weekly * sigma_m_weekly

# Annualize M² (convert weekly return to annual compounded)
m2_annual = (1.0 + m2_weekly) ** ann_factor - 1.0

# (Optional) Benchmark return for comparison: annualized market mean (compounded)
mean_rm_weekly = float(rm_a.mean())
rm_annual = (1.0 + mean_rm_weekly) ** ann_factor - 1.0

# Print nicely
results = {
    "rf_weekly": rf_period,
    "mean_excess_weekly": mean_excess_weekly,
    "sigma_p_weekly": sigma_p_weekly,
    "beta_p": beta_p,
    "Sharpe_weekly": sharpe_weekly,
    "Sharpe_annualized": sharpe_annual,
    "Treynor_weekly": treynor_weekly,
    "Treynor_annualized": treynor_annual,
    "M2_weekly": m2_weekly,
    "M2_annual_compounded": m2_annual,
    "Market_annual_compounded": rm_annual
}

pd.Series(results)

rf_weekly                   0.001302
mean_excess_weekly          0.002341
sigma_p_weekly              0.014253
beta_p                      0.671039
Sharpe_weekly               0.164263
Sharpe_annualized           1.184520
Treynor_weekly              0.003489
Treynor_annualized          0.181430
M2_weekly                   0.004158
M2_annual_compounded        0.240797
Market_annual_compounded    0.104635
dtype: float64